In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the text data (Alice in Wonderland)
with open('wonderland.txt', 'r', encoding='utf-8') as file:
    text_data = file.read().lower().replace("\n", " ")


# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
total_words = len(tokenizer.word_index) + 1


In [3]:
# Create input sequences and labels for training
input_sequences = []
for line in text_data.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)


In [4]:
# Build and train the SimpleRNN model
model_rnn = Sequential()
model_rnn.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(total_words, activation='softmax'))
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_rnn.fit(X, y, epochs=10, verbose=0)

In [5]:
# Build and train the GRU model
model_gru = Sequential()
model_gru.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_gru.add(GRU(100))
model_gru.add(Dense(total_words, activation='softmax'))
model_gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_gru.fit(X, y, epochs=10, verbose=0)

In [6]:
# Build and train the LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(total_words, activation='softmax'))
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X, y, epochs=10, verbose=0)

In [7]:
# Function to generate text using the trained models
def generate_text(seed_text, model, max_sequence_len, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [11]:
# Example of generating text with each model
generated_text_rnn = generate_text("alice in Wonderland", model_rnn, max_sequence_length, num_words=20)
generated_text_gru = generate_text("alice in Wonderland", model_gru, max_sequence_length, num_words=20)
generated_text_lstm = generate_text("alice in Wonderland", model_lstm, max_sequence_length, num_words=20)

print("Generated Text (SimpleRNN):", generated_text_rnn)
print("Generated Text (GRU):", generated_text_gru)
print("Generated Text (LSTM):", generated_text_lstm)

1/1 [==============================] - 0s 110ms/step
Generated Text (SimpleRNN): alice in Wonderland “the dear ” said the king “and i don’t get ” said the king “and i don’t get ” said
Generated Text (GRU): alice in Wonderland of the house and she had been to get out of the house and she went on to the door
Generated Text (LSTM): alice in Wonderland down in the sea and she had not been to be a little opportunity to be a little opportunity to
